# Projeto Final


In [20]:
import pandas as pd
import requests
from plyer import notification
import sqlite3
from datetime import datetime

# Função Alerta

In [25]:
def alerta(nivel, base, etapa):
    
    now = str(datetime.now())

    msg = f"Sucesso no carregamento da base {base} na etapa {etapa}.\n{now}"
    msg1 = f"Falha no carregamento da base {base} na etapa {etapa}.\n{now}"
    


    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel  == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print("Nivel",nivel,"não disponível!")
        
    
    if response.status_code == 200:
        notification.notify(
        title=title,
        message=msg,
        app_name='alerta',
        timeout=10
        )
    else:
        notification.notify(
        title=title,
        message=msg1,
        app_name='alerta',
        timeout=10
        )
    

# Função Banco de Dados

In [45]:
def salva_bd(df, nome_tabela):
    conn = sqlite3.connect('coderhouse.db')
    df.to_sql(nome_tabela, conn, if_exists='replace', index=False)
    conn.close()
    
def carrega_bd(nome_tabela):
    conn = sqlite3.connect('coderhouse.db')
    query = f"SELECT * FROM {nome_tabela}"
    df = pd.read_sql(query, conn)
    conn.close()
    

# Extraindo Dados da API

API Bancos

In [29]:
url = "https://brasilapi.com.br/api/banks/v1"
response = requests.get(url)
alerta(nivel=1,
       base="Bancos",
       etapa="Extração")

API Corretoras

In [30]:
url_1 = "https://brasilapi.com.br/api/cvm/corretoras/v1"
response_1 = requests.get(url_1)
alerta(nivel=1,
       base="Corretoras",
       etapa="Extração")

API UF

In [31]:
url_2 = "https://brasilapi.com.br/api/ibge/uf/v1"
response_2 = requests.get(url_2)
alerta(nivel=1,
       base="UF",
       etapa="Extração")

# Análise e Transformação

API Bancos

In [ ]:
df_bancos = pd.DataFrame(response.json())
df_bancos.isna().sum()
df_bancos[df_bancos.code.isna()]
df_bancos.dropna(subset=['code', 'name', 'fullName'], inplace=True)
df_bancos.rename(columns={'name':'nome', 'code':'codigo','fullName':'nomeCompleto'}, inplace=True)
df_bancos

API Corretoras

In [ ]:
df_corretoras = pd.DataFrame(response_1.json())
df_corretoras.isna().sum()
df_corretoras.rename(columns={'type':'tipo'}, inplace=True)
df_corretoras


API UF

In [50]:
df_uf = pd.DataFrame(response_2.json())
df_uf.drop(columns=['id','regiao'],inplace=True)

# Salvando e Carregando os DF

Bancos

In [ ]:
salva_bd(df=df_bancos,nome_tabela="Bancos")
alerta(nivel=1,
       base="Bancos",
       etapa="Extraçã")
carrega_bd(nome_tabela="Bancos")
df_bancos.head()


Corretoras

In [ ]:
salva_bd(df=df_corretoras,nome_tabela="Corretoras")
alerta(nivel=1,
       base="Corretoras",
       etapa="Extraçã")
carrega_bd(nome_tabela="Corretoras")
df_corretoras.head()

UF

In [ ]:
salva_bd(df=df_uf,nome_tabela="UF")
alerta(nivel=1,
       base="UF",
       etapa="Extraçã")
carrega_bd(nome_tabela="UF")
df_uf.head()